# 處理APK檔案,取XML資料

In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import os
import shutil
import numpy as np

In [2]:
#將XML轉成非亂碼 , 利用 AXMLPrinter2
def xml_decode(join_folder):
    for apk in os.listdir(join_folder):
        xml_place=os.path.join(join_folder, apk , "AndroidManifest.xml")
        newname=os.path.join(join_folder, apk ,'AndroidManifest_new.xml')
        os.chdir("D:\\")
        command = 'java -jar %s %s > %s'%('AXMLPrinter2.jar',xml_place,newname)
        if(os.system(command) == 0):
            continue
        else:
            print('失败:',command)
    print("Done")

base_path='D:\EC_T\EC_T\HW10\Benign'
#in_folder='Adware_unzip'
#join_folder=os.path.join(base_path,in_folder)
xml_decode(base_path)

Done


In [4]:
# in_folder的位置, 因資料放置不同而有所改變

base_path='D:\EC_T\EC_T\HW10\Benign'
#in_folder='Benign_unzip'
#join_folder=os.path.join(base_path,in_folder)

os.chdir(base_path)

data={}
feature=[]
max_len=-1
_id= 1 

#取出XML檔案的permisson特徵

for apk in os.listdir(base_path):
    
    try:
        xml_place = os.path.join(base_path , apk, "AndroidManifest_new.xml")
        tree = ET.parse(xml_place)
        root = tree.getroot()

        temp=[""]*100
        i=0
        for ele in root.iter('uses-permission'):
            for p in ele.attrib.values():
                temp[i]=p
                feature.append(p)
                i+=1
        data [apk] = temp
        _id+=1
        
    #若無法存取 則將無法存取的資料夾移去Fail的目錄
    except:
        source = os.path.join(base_path , apk)
        in_folder_2 ='Fail'
        destination=os.path.join(base_path,in_folder_2)
        shutil.move(source,destination)

In [5]:
#轉為pd , 儲存特徵為CSV
data_list = data
my_df = pd.DataFrame(data_list)
my_df.to_csv( 'D:\EC_T\EC_T\HW10\Benign_feature.csv'  , index=False )

## 特徵資料前處理

In [82]:
def read_csv (path):
    file_pd = pd.read_csv(path)
    return file_pd

In [83]:
def isNAN(string):
    return string != string

In [84]:
def create_feature_pd(sample):
    feature = []
    # 單獨取出特徵 , 建立 index =apk ,columns =feature 的dataframe
    for apk in sample.columns:
        for i in sample[apk].values:
            if  not isNAN(i):
                feature.append(i)
            else:
                continue
    unique_feature = set (feature) # 去除重複的特徵
    feature_pd = pd.DataFrame(index = sample.columns, columns = unique_feature)
    
    # 將apk有的特徵轉為 1
    for apk in feature_pd.index:
        for feature in sample[apk].values:
            if not isNAN(feature):
                for i in feature_pd.columns:
                    if feature == i :
                        feature_pd.loc[apk,i] = 1
                    #else:
                    #    feature_pd.loc[apk,i] = 0
            #else:
             #   feature_pd.loc[apk,i] = 0
            
    feature_pd = feature_pd.fillna(0)           
    return feature_pd

In [32]:
SMS_feature = read_csv ('D:\EC_T\EC_T\HW10\SMS_feature.csv')
SMS_feature_pd = create_feature_pd ( SMS_feature )
SMS_feature_pd.to_csv( 'D:\EC_T\EC_T\HW10\SMS_feature_pd.csv'  , index=False )

In [33]:
Banking_feature = read_csv ('D:\EC_T\EC_T\HW10\Banking_feature.csv')
Banking_feature_pd = create_feature_pd ( Banking_feature )
Banking_feature_pd.to_csv( 'D:\EC_T\EC_T\HW10\Banking_feature_pd.csv'  , index=False )

In [34]:
Adware_feature = read_csv ('D:\EC_T\EC_T\HW10\Adware_feature.csv')
Adware_feature_pd = create_feature_pd ( Adware_feature )
Adware_feature_pd.to_csv( 'D:\EC_T\EC_T\HW10\Adware_feature_pd.csv'  , index=False )

#### 二類分類

In [35]:
Ma_feature = pd.concat([SMS_feature_pd,Banking_feature_pd,Adware_feature_pd], join='outer')
Ma_feature = Ma_feature.fillna(0)
Ma_feature ['Lable'] = 'Malicios'
Ma_feature.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6403 entries, 000a067df9235aea987cd1e6b7768bcc1053e640b267c5b1f0deefc18be5dbe1 to ffcab4fa2a51f5fe67df821c0fa5c757d6d59164f5168dcc8bb4fbb739d73099
Columns: 467 entries, android.permission.INTERNET to Lable
dtypes: float64(376), int64(90), object(1)
memory usage: 22.9+ MB


In [36]:
Ma_feature.to_csv( 'D:\EC_T\EC_T\HW10\Ma_feature.csv'  , index=False )

In [ ]:
Ma_feature = read_csv ('D:\EC_T\EC_T\HW10\Ma_feature.csv')

In [39]:
Benign_feature = read_csv ('D:\EC_T\EC_T\HW10\Benign_feature.csv')
Benign_feature_pd = create_feature_pd ( Benign_feature )

Benign_feature_pd ['Lable'] = 'Benign'
Benign_feature_pd.to_csv( 'D:\EC_T\EC_T\HW10\Benign_feature_pd.csv'  , index=False )

In [40]:
Total_feature = pd.concat([Benign_feature_pd,Ma_feature], join='outer')
Total_feature = Total_feature.fillna(0)
Total_feature.to_csv( 'D:\EC_T\EC_T\HW10\Total_feature.csv' , index = False)

#### 多類別分類

In [85]:
SMS_feature_pd = read_csv ('D:\EC_T\EC_T\HW10\SMS_feature_pd.csv')
Banking_feature_pd = read_csv ('D:\EC_T\EC_T\HW10\Banking_feature_pd.csv')
Adware_feature_pd = read_csv ('D:\EC_T\EC_T\HW10\Adware_feature_pd.csv')
Benign_feature_pd = read_csv ('D:\EC_T\EC_T\HW10\Benign_feature_pd.csv')

In [86]:
SMS_feature_pd ['Lable']='SMS'
Banking_feature_pd ['Lable']='Banking'
Adware_feature_pd ['Lable']='Adware'

In [87]:
Muti_feature = pd.concat([SMS_feature_pd,Banking_feature_pd,Adware_feature_pd,Benign_feature_pd], join='outer')
Muti_feature = Muti_feature.fillna(0)
Muti_feature.to_csv( 'D:\EC_T\EC_T\HW10\Muti_feature.csv' , index = False)

### 資料轉為圖片

In [2]:
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from PIL import Image
import random

feature = pd.read_csv('D:\EC_T\EC_T\HW10\Total_feature.csv')
feature.head()

,net.infojobs.mobile.android.permission.C2D_MESSAGE,com.singlegateway.guidegame2.permission.C2D_MESSAGE,com.infostretch.iSOSAndroid.permission.C2D_MESSAGE,com.inditex.ecommerce.bershka.permission.C2D_MESSAGE,android.permission.CALL_PHONE,com.frogsparks.mytrails.gcm.permission.C2D_MESSAGE,com.sonymobile.usbux.permission.USB_HOST_DETECTION,com.petrocanada.my_petro_canada.permission.C2D_MESSAGE,com.m3tech.ablamc.permission.C2D_MESSAGE,android.permission.SEND_MAIL,...,gexin.permission.GexinService,adnroid.permission.ACCESS_CHECKIN_PROPERTTES,android.launcher.permission.READ_SETTINGS,android.permission.ACCES_MOCK_LOCATION,hitask.permission.FLOATWIN_SERVICE,adnroid.permission.CHANGE_WIFI_STATE,com.timsu.astrid.permission.READ_TASKS,android.intent.category.HOME,android.permission.CHANGE_WIMAX_STATE,android.permission.PERMISSION_NAME
0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
feature ['Lable']=feature['Lable'].replace('Benign','1')
feature ['Lable']=feature['Lable'].replace('Malicios','2')    

In [13]:
test = feature
total =3

for i in test.index :
    for j in test.columns:
        if test.loc[i,j] == 0:
            if total !=256:
                test.loc[i,j]= total
                total+=1
            else:
                total=3
            
test.head()

,net.infojobs.mobile.android.permission.C2D_MESSAGE,com.singlegateway.guidegame2.permission.C2D_MESSAGE,com.infostretch.iSOSAndroid.permission.C2D_MESSAGE,com.inditex.ecommerce.bershka.permission.C2D_MESSAGE,android.permission.CALL_PHONE,com.frogsparks.mytrails.gcm.permission.C2D_MESSAGE,com.sonymobile.usbux.permission.USB_HOST_DETECTION,com.petrocanada.my_petro_canada.permission.C2D_MESSAGE,com.m3tech.ablamc.permission.C2D_MESSAGE,android.permission.SEND_MAIL,...,gexin.permission.GexinService,adnroid.permission.ACCESS_CHECKIN_PROPERTTES,android.launcher.permission.READ_SETTINGS,android.permission.ACCES_MOCK_LOCATION,hitask.permission.FLOATWIN_SERVICE,adnroid.permission.CHANGE_WIFI_STATE,com.timsu.astrid.permission.READ_TASKS,android.intent.category.HOME,android.permission.CHANGE_WIMAX_STATE,android.permission.PERMISSION_NAME
0,3.0,4.0,5.0,6.0,7,8.0,9.0,10.0,11.0,12.0,...,242.0,243.0,244.0,245.0,246.0,247.0,248.0,249.0,250.0,251.0
1,252.0,253.0,254.0,255.0,16,3.0,4.0,5.0,6.0,7.0,...,234.0,235.0,236.0,237.0,238.0,239.0,240.0,241.0,242.0,243.0
2,244.0,245.0,246.0,247.0,248,249.0,250.0,251.0,252.0,253.0,...,217.0,218.0,219.0,220.0,221.0,222.0,223.0,224.0,225.0,226.0
3,227.0,228.0,229.0,230.0,231,232.0,233.0,234.0,235.0,236.0,...,203.0,204.0,205.0,206.0,207.0,208.0,209.0,210.0,211.0,212.0
4,213.0,214.0,215.0,216.0,217,218.0,219.0,220.0,221.0,222.0,...,179.0,180.0,181.0,182.0,183.0,184.0,185.0,186.0,187.0,188.0


In [18]:
test.to_csv("test.csv",index=False)

In [19]:
img_data =pd.read_csv('test.csv')

In [74]:
end = len(img_data.columns)-1
w = int(end**0.5) + 1

data_list = []
num = 1 
for i in img_data.index:
    val = list(img_data.iloc[i].values)

    test = [val[i:i+w] for i in range(0,end,w)]
    test [-1] = test[-1] + ([0]*(w-len(test[-1])))

    img  = Image.fromarray(np.array(test , dtype = 'int8')).convert('L')
    
    if img_data.loc[ i , 'Lable'] == '1':
        file_name = 'Benign_'+str(num) + '.jpg'
    else:
        file_name = 'Malicious_'+str(num) + '.jpg'
    
    path = os.path.join( 'D:\EC_T\EC_T\HW10\CNN_Image',file_name)
    img.save(path)
    
    num+=1

# Model Traning

### SVM model

In [2]:
data = pd.read_csv('D:\EC_T\EC_T\HW10\Total_feature.csv')
#muti_data = pd.read_csv('D:\EC_T\EC_T\HW10\Muti_feature.csv')

In [4]:
from sklearn import svm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split ,cross_val_score

In [4]:
# 2_label
X = data.drop(['Lable'],axis=1)
y = data['Lable']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42)

In [8]:
clf=svm.SVC(kernel='rbf',C=1,gamma='auto')
clf.fit(X_train,y_train)
accu = cross_val_score(clf, X_test, y_test, cv=10)
print('Accu : ',max(accu))

Accu :  0.8405797101449275


In [9]:
# 二元分類
clf2=svm.SVC(kernel='linear',C=1,gamma='auto')
clf2.fit(X_train,y_train)
accu = cross_val_score(clf2, X_test, y_test, cv=10)
print('Accu : ',max(accu))

Accu :  0.9759615384615384


In [11]:
clf3=svm.SVC(kernel='sigmoid',C=1,gamma='auto')
clf3.fit(X_train,y_train)
accu = cross_val_score(clf3, X_test, y_test, cv=10)
print('Accu : ',max(accu))

Accu :  0.6057692307692307


### RF model

In [5]:
rf_data=data

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
rf_data['Lable'] = labelencoder.fit_transform(rf_data['Lable'])

X = rf_data.drop(['Lable'],axis=1)
y = rf_data['Lable']
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42)

In [6]:
# 二元分類
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 100, n_jobs=4, max_features='auto')
rf.fit(X_train, y_train)
accu = cross_val_score(rf, X_test, y_test, cv=10)
print('Accu : ',max(accu))

Accu :  0.9402678728469891


#### 做資料刪減--改善準確率

In [52]:
total = 0
for i in rf_data['Lable']:
    if i == 0:
        total+=1
print ('Benign num: ',total)
print ('Malicious num: ',len(rf_data['Lable'])-total)
print ('差 :' , len(rf_data['Lable'])-2*total)

Benign num:  3975
Malicious num:  6403
差 : 2428


In [38]:
data_reduce = data.T
data_reduce = data_reduce.drop(labels=range(8377,10378),axis=1)
data_reduce = data_reduce.T

In [39]:
X3 = data_reduce.drop(['Lable'],axis=1)
y3 = data_reduce['Lable']
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3,test_size=0.2,random_state=42)

In [41]:
#做資料刪減後
rf.fit(X3_train, y3_train)
accu = cross_val_score(rf, X3_test, y3_test, cv=10)
print('Accu : ',max(accu))

Accu :  0.9596470456924532
